In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/nlp-getting-started/sample_submission.csv
/kaggle/input/nlp-getting-started/train.csv
/kaggle/input/nlp-getting-started/test.csv


In [3]:
pd.set_option('display.max_colwidth', None)
df = pd.read_csv('../input/nlp-getting-started/train.csv')
df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation orders in California",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [4]:
df.count()

id          7613
keyword     7552
location    5080
text        7613
target      7613
dtype: int64

#### There are few missing values in 'keyword', and 'location'

In [5]:
df.describe()

,id,target
count,7613.000000,7613.00000
mean,5441.934848,0.42966
std,3137.116090,0.49506
min,1.000000,0.00000
25%,2734.000000,0.00000
50%,5408.000000,0.00000
75%,8146.000000,1.00000
max,10873.000000,1.00000


In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7613 entries, 0 to 7612
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   id        7613 non-null   int64 
 1   keyword   7552 non-null   object
 2   location  5080 non-null   object
 3   text      7613 non-null   object
 4   target    7613 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 297.5+ KB


In [7]:
df['keyword'].unique()
df.loc[df['keyword'] == 'ambulance'].head() # to display the top 5 'ambulance' keyword-ed tweets

,id,keyword,location,text,target
171,246,ambulance,NaN,Early wake up call from my sister begging me to come over &amp; ride w/her in the ambulance to the hospital #RODKiai,1
172,247,ambulance,Jackson,http://t.co/AY6zzcUpnz Twelve feared killed in Pakistani air ambulance helicopter crash http://t.co/sC9dNS41Mc,1
173,248,ambulance,New York / Worldwide,Two air ambulances on scene of serious crash between two cars and lorry in ... - http://t.co/9pFEaQeSki http://t.co/fntG70rnkx | #EMSNeÛ_,1
174,249,ambulance,NaN,Twelve feared killed in Pakistani air ambulance helicopter crash - Reuters http://t.co/mDnUGVuBwN #yugvani,1
175,251,ambulance,"New Orleans, LA",Leading emergency services boss welcomes new ambulance charity http://t.co/Mj2jQ2pSv6,0


### The keyword, location, and id would not give an accurate estimation of the nature of tweets, so I remove them

In [8]:
df = df.drop(['id', 'keyword', 'location'], axis = 1)
df.head()

,text,target
0,Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all,1
1,Forest fire near La Ronge Sask. Canada,1
2,All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected,1
3,"13,000 people receive #wildfires evacuation orders in California",1
4,Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school,1


In [19]:
import re
import nltk.corpus
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split as tts

### Writing a small function to clean the text by removing the special characters

In [14]:
def clean_df(df, text):
    df[text] = df[text].str.lower()
    df[text] = df[text].apply(
        lambda elem: re.sub(r"(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)|^rt|http.+?", "", elem)
    )
    return df

In [17]:
cleaned_df = clean_df(df, "text")
cleaned_df.head()

,text,target
0,our deeds are the reason of this earthquake may allah forgive us all,1
1,forest fire near la ronge sask canada,1
2,all residents asked to shelter in place are being notified by officers no other evacuation or shelter in place orders are expected,1
3,13000 people receive wildfires evacuation orders in california,1
4,just got sent this photo from ruby alaska as smoke from wildfires pours into a school,1


### Removing the stopwords using NLTK.
#### Stopwords are those words which do not 'mean' anything with respect to the text.

In [22]:
nltk.download('stopwords')
stop = stopwords.words('english')

cleaned_df['text'] = cleaned_df['text'].apply(
    lambda x: ' '.join([word for word in x.split() if word not in (stop)])
)

cleaned_df.head()

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
